In [4]:
import numpy as np
from scipy.optimize import minimize

W_i = np.array([1/100, 1/100, 1/100, 1/100])
D = np.array([1/w_i for w_i in W_i])
x0 = D
A = np.array([[1000, 2584],
            [0, 4],
            [25, 354],
            [102, 878]])
B = np.array([100000, 450002])

def linear_method(w, d):
    return ((w/d-1)**2)

def ranking_ratio_method(w, d):
    return (w/d)*np.log(w/d)-(w/d)+1

# def objective(W):
#     return (1/2)*sum(
#         d_k*linear_method(w_k, d_k) for w_k, d_k in zip(W, D)
#     )

def objective(W):
    return sum(
        d_k*ranking_ratio_method(w_k, d_k) for w_k, d_k in zip(W, D)
    )

def constraint(W):
    return A.T@W-B

constraints = {"type" : "eq", "fun" : constraint}

result = minimize(objective, 
                  x0=x0, 
                  method="trust-constr", 
                  constraints=constraints)

In [45]:
import numpy as np
from scipy.optimize import minimize

W_i = np.array([1/100, 1/100, 1/100, 1/100])
A = np.array([[1000, 2584],
            [0, 4],
            [25, 354],
            [102, 878]])
B = np.array([100000, 450002])

class MarginCalibration:

    def __init__(
        self,
        sampling_probabilities,
        calibration_matrix,
        calibration_target,
        calibration_method,
        lower_bound=None,
        upper_bound=None
    ):

        self.sampling_probabilities = sampling_probabilities
        self.calibration_matrix = calibration_matrix
        self.calibration_target = calibration_target
        self.calibration_method = calibration_method
        self.lower_bound=lower_bound
        self.upper_bound=upper_bound

    def initialize_sampling_weights(self):
        return np.array([1 / prob_i for prob_i in self.sampling_probabilities])

    def _linear_method(self, w, d):
        return (w / d - 1) ** 2

    def _ranking_ratio_method(self, w, d):
        return (w / d) * np.log(w / d) - (w / d) + 1

    def _logit_method(self, w, d):
        x=w/d
        a = (x-self.lower_bound)*np.log((x-self.lower_bound)/(1-self.lower_bound))
        b = (self.upper_bound-x)*np.log((self.upper_bound-x)/(self.upper_bound-1))
        c = (self.upper_bound-self.lower_bound)/((1-self.lower_bound)*(self.upper_bound-1))
        return (a+b)/c

    def initialize_method(self):

        dict_method = {
            "linear": self._linear_method,
            "ranking_ratio": self._ranking_ratio_method,
            "truncated_linear": self._linear_method,
            "logit": self._logit_method,
        }
        try:
            return dict_method[self.calibration_method]

        except KeyError:
            raise ValueError(
                f"""Invalid value : {self.calibration_method}. 
                Must be one of : 'linear', 'ranking_ratio', 'truncated_linear'"""
            )

    def objective(self, calibration_weights):

        sampling_weights = self.initialize_sampling_weights()

        return sum(
            d_k * self.initialize_method()(w_k, d_k)
            for w_k, d_k in zip(calibration_weights, sampling_weights)
        )

    def constraint(self, calibration_weights):
        return self.calibration_matrix.T @ calibration_weights - self.calibration_target

    def calibration(self):

        constraints = {"type": "eq", "fun": self.constraint}

        x0 = self.initialize_sampling_weights()

        if self.calibration_method in ["truncated_linear", "logit"]:
            if isinstance(self.lower_bound, (int, float)) and isinstance(
                self.upper_bound, (int, float)
            ):
                if (self.lower_bound < 1) and (self.upper_bound > 1):
                    sampling_weights = self.initialize_sampling_weights()
                    bounds = [
                        (self.lower_bound * d_k, self.upper_bound * d_k)
                        for d_k in sampling_weights
                    ]
                else:
                    raise ValueError(
                        """The lower bound should be strictly inferior to 1, 
                        the upper bound strictly superior to 1"""
                    )
            else:
                raise TypeError(
                    """'lower_bound' and 'upper_bound' must be numeric values
                    when using 'truncated_linear' or 'logit' methods"""
                )
        else:
            bounds = None

        return minimize(
            self.objective,
            x0=x0,
            method="trust-constr",
            constraints=constraints,
            bounds=bounds,
        )

In [46]:
MarginCalibration(W_i, A, B, "linear").calibration().x

array([ 72.03355474, 100.89117545, 163.77747608, 234.03929763])

In [47]:
MarginCalibration(W_i, A, B, "ranking_ratio").calibration().x

array([ 71.7634369 , 100.59251627, 152.06344919, 239.55859679])

In [48]:
MarginCalibration(W_i, A, B, "truncated_linear", 0.9, 1.5).calibration().x

array([ 72.63096179, 155.25181912, 188.11562062, 222.22043154])

In [49]:
MarginCalibration(W_i, A, B, "logit", 0.9, 1.5).calibration().x

/tmp/ipykernel_928/2275174773.py:41: RuntimeWarning: invalid value encountered in log
  a = (x-self.lower_bound)*np.log((x-self.lower_bound)/(1-self.lower_bound))
/tmp/ipykernel_928/2275174773.py:42: RuntimeWarning: invalid value encountered in log
  b = (self.upper_bound-x)*np.log((self.upper_bound-x)/(self.upper_bound-1))


ValueError: array must not contain infs or NaNs